In [1]:
import os
import sys
import json
import tqdm
from transformers import AutoTokenizer

In [2]:
data_path = "processed"

In [3]:
with open(os.path.join(data_path, "train_raw.jsonl")) as f:
    train_raw_data = f.readlines()
    
with open(os.path.join(data_path, "val_raw.jsonl")) as f:
    val_raw_data = f.readlines()
    
with open(os.path.join(data_path, "test_raw.jsonl")) as f:
    test_raw_data = f.readlines()

In [4]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [5]:
def process_data(data):
    all_dialog_pairs = []

    for line in tqdm.tqdm(data):
        history_turns = []
        dialog = json.loads(line)

        for i, turn in enumerate(dialog):

            if i==0:
                history_turns.append(turn)
                continue

            source = "</s>".join(history_turns)
            target = turn

            example = {
                "source": source,
                "target": target
            }

            tokens = tokenizer.encode(source + target)
            
            if len(tokens) > 512:
                break
            
            all_dialog_pairs.append(example)

            # post
            history_turns.append(turn)
            
    return all_dialog_pairs

In [6]:
train_data = process_data(train_raw_data)
val_data = process_data(val_raw_data)
test_data = process_data(test_raw_data)

 70%|███████   | 702/1000 [00:01<00:00, 542.28it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (520 > 512). Running this sequence through the model will result in indexing errors
100%|██████████| 1000/1000 [00:01<00:00, 554.79it/s]


In [7]:
with open("train.jsonl", "w") as f:
    for line in train_data:
        f.write(json.dumps(line))
        f.write("\n")
        
with open("val.jsonl", "w") as f:
    for line in val_data:
        f.write(json.dumps(line))
        f.write("\n")
        
with open("test.jsonl", "w") as f:
    for line in test_data:
        f.write(json.dumps(line))
        f.write("\n")